# 1) Large size vectors: lcc MAD-Mex

## a) Normalize, reproject to wgs84

In [1]:
import unicodedata

from pyproj import Proj

import fiona
from fiona.crs import to_string
from rasterio.warp import transform_geom
from rasterio.crs import CRS as CRS_rio

In [2]:
#direc = "/LUSTRE/MADMEX/products/landcoverchange/landsat/mosaics/2017_2018"
direc = "/LUSTRE/MADMEX/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/HIDALGO/"

In [3]:
#input_filename = direc + "/madmex_landsat_changes_2017-2018.shp"
#output_filename = direc + "/madmex_landsat_changes_2017-2018_wgs84_using_fiona.shp"
#layer = "madmex_landsat_changes_2017-2018_wgs84_using_fiona"
input_filename = direc + "HIDALGO_merge.shp"
output_filename = direc + "HIDALGO_merge_wgs84_using_fiona2.shp"
layer = "HIDALGO_merge_wgs84_using_fiona2"
dst_crs = "4326"
name_attribute1 = "t1_dsc_31"
name_attribute2 = "t2_dsc_31"
name_attribute3 = "t1_dsc_17"
name_attribute4 = "t2_dsc_17"
name_attribute5 = "cmb_dsc_31"
name_attribute6 = "cmb_dsc_17"

In [4]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6]

In [5]:
def reproj_geom(geom, source_crs, destiny_crs):
    return transform_geom(CRS_rio.from_proj4(source_crs), CRS_rio.from_epsg(destiny_crs),
                          geom)

In [6]:
def normalize_name_classes(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII','ignore').decode('utf-8')

In [7]:
def wrapper_reproj_normalize_and_write_vector(feature_collection, feature_collection_schema,
                                              source_crs, destiny_crs,
                                              output_filename,
                                              is_geographic = True):
    dst = fiona.open(output_filename, 'w',
                     driver = 'ESRI Shapefile',
                     layer = layer,
                     crs = CRS_rio.from_epsg(dst_crs).to_proj4(),
                     schema = feature_collection_schema)
    dst.close()
    with fiona.open(output_filename, 'a',
                    driver = 'ESRI Shapefile',
                    layer = layer,
                    crs = CRS_rio.from_epsg(destiny_crs).to_proj4(),
                    schema = feature_collection_schema) as dst:
        for feature in feature_collection:
            if not is_geographic:
                feature['geometry'] = reproj_geom(feature['geometry'],
                                                  source_crs,
                                                  destiny_crs)
            for att in list_name_attributes:
                feature['properties'][att] = normalize_name_classes(feature['properties'][att])
            dst.write(feature)

In [8]:
with fiona.open(input_filename) as src:
    fc = (feature for feature in src if not None in [feature['properties'][att] for att in list_name_attributes] and
                                                     feature['geometry'] is not None)
    src_crs = to_string(src.crs)
    proj_src_crs = Proj(src_crs)
    fc_schema = src.schema
    if not proj_src_crs.crs.is_geographic:
        wrapper_reproj_normalize_and_write_vector(fc, fc_schema,
                                                  src_crs, 
                                                  dst_crs,
                                                  output_filename,
                                                  is_geographic = False)
    else:
        wrapper_reproj_normalize_and_write_vector(fc, fc_schema, 
                                                  src_crs, dst_crs,
                                                  output_filename)
        

## b) Register large vector layer in geonode via `shp2pgsql` and `updatelayers`

**Make sure stack deployment of geonode is up and running**

**1) Create `~/.pgpass` with contents: `hostname:port:database:username:password` and `chmod 0600 ~/.pass`**

In [9]:
import subprocess

In [37]:
#next need to be in lowercase
name_table = "hidalgo_madmex_lcc_2017_2018_fiona" #madmex_landsat_2017-2018_lcc


In [21]:
l_cmd1 = ["shp2pgsql",
          output_filename,
          name_table,
          "public." + name_table + ".shp"]

In [22]:
p1 = subprocess.Popen(l_cmd1, stdout=subprocess.PIPE)

In [23]:
l_cmd2 = ["psql",
          "-q",
          "-h",
          "geonode.conabio.gob.mx",
          "-d",
          "geonode_data",
          "-U", 
          "geonode"]

In [24]:
p2 = subprocess.Popen(l_cmd2, stdin=p1.stdout, stderr=subprocess.PIPE)

In [25]:
p2.communicate()

(None, b'')

In [26]:
p2.poll()

0

In [27]:
p1.terminate()

In [28]:
p2.terminate()

**2) Add to geoserver from geonode_data database**. Need to follow: https://training.geonode.geo-solutions.it/004_admin_workshop/007_loading_data_into_geonode/geoserver.html

**3) Update layers using docker container `spcgeonode_django_1`**

In [29]:
from docker import APIClient

In [30]:
c = APIClient(base_url='unix://var/run/docker.sock')

In [31]:
c

In [43]:
cmd = "".join(["python manage.py updatelayers -s geonode_data -w geonode -f ",
               name_table, " ",
               "--settings=geonode.local_settings"])

In [50]:
cmd

'python manage.py updatelayers -s geonode_data -w geonode -f hidalgo_madmex_lcc_2017_2018_fiona --settings=geonode.local_settings'

In [46]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)

In [47]:
ex_start = c.exec_start(exec_id=ex, stream=True)

**To retrieve output:**

In [49]:
#for val in ex_start:
#    print(val)

**4) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# Large size raster: lc MAD-Mex

## a) Reproject to wgs84, compress and tile it

Based on: 

https://rasterio.readthedocs.io/en/latest/topics/reproject.html

https://rasterio.readthedocs.io/en/latest/topics/image_options.html

In [1]:
import numpy as np

from pyproj import Proj
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/31'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Aguascalientes/31/'
direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Hidalgo/31/'

In [3]:
#input_filename = ''.join([direc, '/madmex_sentinel2_2017_31.tif'])
#output_filename = ''.join([direc, '/madmex_sentinel2_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Aguascalientes_2017_31.tif'])
#output_filename = ''.join([direc, '/Aguascalientes_2017_31_wgs84_tiled_rasterio.tif'])
input_filename = ''.join([direc, '/Hidalgo_2017_31.tif'])
output_filename = ''.join([direc, '/Hidalgo_2017_31_wgs84_tiled_rasterio.tif'])
dst_crs = 'EPSG:4326'

In [4]:
def wrapper_reproj_and_write(source_dataset, destiny_crs, output_filename,
                             is_geographic = True):
    source_meta = source_dataset.meta.copy()
    source_transform = source_dataset.transform
    source_width = source_dataset.width
    source_height = source_dataset.height
    source_crs = source_dataset.crs
    if not is_geographic:
        transform, width, height = calculate_default_transform(source_crs, destiny_crs, 
                                                               source_width, source_height, 
                                                               *source_dataset.bounds)
        source_meta.update({'crs': destiny_crs,
                           'transform': transform,
                           'width': width,
                           'height': height
                            })
    else:
        transform, width, height = source_transform, source_width, source_height
    
    source_meta.update({'driver': 'GTiff',
                        'count': 1,
                        'dtype': rasterio.uint8,
                        'compress': 'lzw',
                        'tiled': True
                        })
    with rasterio.open(output_filename, 'w', **source_meta,
                       ) as dst:
        array = np.zeros((height, width), dtype=rasterio.uint8)
        reproject(source=rasterio.band(source_dataset, 1),
                  destination=array,
                  src_transform=source_transform,
                  src_crs=source_crs,
                  dst_transform=transform,
                  dst_crs=dst_crs,
                  resampling=Resampling.nearest
                  )  
        dst.write(array, 1)

In [5]:
with rasterio.open(input_filename) as src:
    src_crs = src.crs.to_string()
    proj_crs = Proj(src_crs)
    if not proj_crs.crs.is_geographic:
        wrapper_reproj_and_write(src, dst_crs, output_filename,
                                 is_geographic=False)
    else:
        wrapper_reproj_and_write(src, dst_crs, output_filename)

## b) Register large raster layer in geonode via `importlayers`

**1) Importlayers cmd**

In [28]:
from docker import APIClient

In [29]:
c = APIClient(base_url='unix://var/run/docker.sock')

In [30]:
c

In [31]:
region = "".join(["\"Hidalgo, Mexico, North America, Latin America\""])
name = "\"sentinel2_Hidalgo_2017_31_tiled\""
title = "\"sentinel2_Hidalgo_2017_31_tiled\""
abstract = "\"Sentinel2 MAD-Mex land cover classification\""
kw = "\"MAD-Mex, Sentinel2, GeoTIFF, WCS\""


In [32]:
cmd = "".join(["python manage.py importlayers -v 3 -i -o ",
               "-n ", name, " ",
               "-t ", title, " ",
               "-a ", abstract, " ",
               "-k ", kw, " ",
               "-r ", region, " ",
               output_filename, " ",
               "--settings=geonode.local_settings"])

In [33]:
cmd

'python manage.py importlayers -v 3 -i -o -n "sentinel2_Hidalgo_2017_31_tiled" -t "sentinel2_Hidalgo_2017_31_tiled" -a "Sentinel2 MAD-Mex lc" -k "MAD-Mex, Sentinel2, GeoTIFF, WCS" -r "Hidalgo Mexico, North America, Latin America" /LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Hidalgo/31//Hidalgo_2017_31_wgs84_tiled_rasterio.tif --settings=geonode.local_settings'

In [34]:
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)

In [35]:
ex_start = c.exec_start(exec_id=ex, stream=True)

**To retrieve output:**

In [24]:
#for val in ex_start:
#    print(val)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# Next is on hold, will be develop

## Small to Medium size vectors

In [1]:
from pyproj import Proj
import geopandas as gpd

In [2]:
direc = '/LUSTRE/MADMEX/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/AGUASCALIENTES'

In [3]:
input_file = direc + '/AGUASCALIENTES_merge.shp'
out_filename = direc + '/AGUASCALIENTES_merge_wgs84_using_geopandas.shp'
layer = 'AGUASCALIENTES_merge_wgs84_using_geopandas'

In [4]:
gdf = gpd.read_file(input_file)

In [5]:
gdf.head()

,id,change,mdmx_t1,mdmx_t2,cmb_clv_31,mdmx_cls,t1_clv_31,t2_clv_31,t1_clv_17,t2_clv_17,...,cmb_prb,cmb_clv_17,cmb_dsc_31,cmb_dsc_17,ipcc_clv,ipcc_dsc,area,oid,id_1,geometry
0,221682,12827,28,27,28-27,1,28.0,27.0,14.0,13.0,...,8.090109,14-13,Tierras Agrícolas - Pastizales,Tierras agrícolas - Pastizales y otra vegetaci...,15.0,CONV GL,0.41,8124,None,"POLYGON ((2397540.943 1164609.003, 2397540.943..."
1,68283,12703,27,3,27-3,1,27.0,3.0,13.0,2.0,...,16.683148,13-2,Pastizales - Bosque de Encino y Bosque de Galería,Pastizales y otra vegetación herbácea - Bosque...,9.0,CONV FL,0.70,4952,None,"POLYGON ((2398330.943 1164604.404, 2398330.943..."
2,68193,11427,14,27,14-27,1,14.0,27.0,7.0,13.0,...,17.413904,7-13,Matorral Crasicaule - Pastizales,Matorral alto denso - Pastizales y otra vegeta...,4.0,FL-GL,0.08,4862,None,"POLYGON ((2398180.943 1164605.278, 2398180.943..."
3,221758,12714,27,14,27-14,1,27.0,14.0,13.0,7.0,...,2.954326,13-7,Pastizales - Matorral Crasicaule,Pastizales y otra vegetación herbácea - Matorr...,9.0,CONV FL,0.20,8200,None,"POLYGON ((2397790.943 1164607.352, 2397824.645..."
4,68182,12703,27,3,27-3,1,27.0,3.0,13.0,2.0,...,16.683148,13-2,Pastizales - Bosque de Encino y Bosque de Galería,Pastizales y otra vegetación herbácea - Bosque...,9.0,CONV FL,0.11,4851,None,"POLYGON ((2401150.943 1164577.352, 2401170.943..."


In [6]:
name_attribute1 = "t1_dsc_31"
name_attribute2 = "t2_dsc_31"
name_attribute3 = "t1_dsc_17"
name_attribute4 = "t2_dsc_17"
name_attribute5 = "cmb_dsc_31"
name_attribute6 = "cmb_dsc_17"

In [7]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6]

In [8]:
gdf[list_name_attributes[0]][gdf[list_name_attributes[0]].isnull()]

74791    None
74794    None
74834    None
74878    None
Name: t1_dsc_31, dtype: object

In [9]:
gdf[list_name_attributes[0]][gdf[list_name_attributes[0]].isna()]

74791    None
74794    None
74834    None
74878    None
Name: t1_dsc_31, dtype: object

In [10]:
gdf[list_name_attributes[1]][gdf[list_name_attributes[0]].isna()]

74791    None
74794    None
74834    None
74878    None
Name: t2_dsc_31, dtype: object

In [11]:
gdf[list_name_attributes[2]][gdf[list_name_attributes[0]].isna()]

74791    None
74794    None
74834    None
74878    None
Name: t1_dsc_17, dtype: object

In [12]:
gdf[list_name_attributes[2]][gdf[list_name_attributes[0]].isna()]

74791    None
74794    None
74834    None
74878    None
Name: t1_dsc_17, dtype: object

In [13]:
gdf[list_name_attributes[3]][gdf[list_name_attributes[0]].isna()]

74791    None
74794    None
74834    None
74878    None
Name: t2_dsc_17, dtype: object

In [14]:
gdf[list_name_attributes[4]][gdf[list_name_attributes[0]].isna()]

74791    None
74794    None
74834    None
74878    None
Name: cmb_dsc_31, dtype: object

In [15]:
gdf[list_name_attributes[5]][gdf[list_name_attributes[0]].isna()]

74791    None
74794    None
74834    None
74878    None
Name: cmb_dsc_17, dtype: object

In [16]:
gdf.iloc[74791]

id                                                       367877
change                                                    10029
mdmx_t1                                                       0
mdmx_t2                                                      29
cmb_clv_31                                                 0-29
mdmx_cls                                                      1
t1_clv_31                                                   NaN
t2_clv_31                                                   NaN
t1_clv_17                                                   NaN
t2_clv_17                                                   NaN
t1_dsc_31                                                  None
t2_dsc_31                                                  None
t1_dsc_17                                                  None
t2_dsc_17                                                  None
cmb_prb                                                     NaN
cmb_clv_17                              

In [17]:
gdf.dropna(subset=list_name_attributes, inplace=True)

In [18]:
gdf.iloc[74791]

id                                                       367851
change                                                    12928
mdmx_t1                                                      29
mdmx_t2                                                      28
cmb_clv_31                                                29-28
mdmx_cls                                                      1
t1_clv_31                                                    29
t2_clv_31                                                    28
t1_clv_17                                                    15
t2_clv_17                                                    14
t1_dsc_31                                   Urbano y Construido
t2_dsc_31                                     Tierras Agrícolas
t1_dsc_17                                   Urbano y construido
t2_dsc_17                                     Tierras agrícolas
cmb_prb                                                 13.1019
cmb_clv_17                              

In [19]:
gdf[list_name_attributes[0]][gdf[list_name_attributes[0]].isna()]

Series([], Name: t1_dsc_31, dtype: object)

In [20]:
gdf[list_name_attributes[1]][gdf[list_name_attributes[0]].isna()]

Series([], Name: t2_dsc_31, dtype: object)

In [21]:
gdf[list_name_attributes[2]][gdf[list_name_attributes[0]].isna()]

Series([], Name: t1_dsc_17, dtype: object)

In [22]:
gdf[list_name_attributes[2]][gdf[list_name_attributes[0]].isna()]

Series([], Name: t1_dsc_17, dtype: object)

In [23]:
gdf[list_name_attributes[3]][gdf[list_name_attributes[0]].isna()]

Series([], Name: t2_dsc_17, dtype: object)

In [24]:
gdf[list_name_attributes[4]][gdf[list_name_attributes[0]].isna()]

Series([], Name: cmb_dsc_31, dtype: object)

In [25]:
gdf[list_name_attributes[5]][gdf[list_name_attributes[0]].isna()]

Series([], Name: cmb_dsc_17, dtype: object)

**Reproject**

In [26]:
crs_src = gdf.crs

In [27]:
crs_src

<Projected CRS: PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_WGS_1 ...>
Name: Lambert_Conformal_Conic
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: World Geodetic System 1984
- Ellipsoid: WGS_84
- Prime Meridian: Greenwich

In [28]:
crs_src.to_string()

'PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_WGS_1984",DATUM["WGS_1984",SPHEROID["WGS_84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'

In [29]:
src_crs = crs_src.to_string()
proj_src_crs = Proj(src_crs)

In [30]:
if not proj_src_crs.crs.is_geographic:
    print('not wgs84')
else:
    print('wgs84')

not wgs84


In [31]:
crs_dst = "EPSG:4326"

In [32]:
gdf_reproj = gdf.to_crs(crs_dst)

In [33]:
gdf_reproj.head()

,id,change,mdmx_t1,mdmx_t2,cmb_clv_31,mdmx_cls,t1_clv_31,t2_clv_31,t1_clv_17,t2_clv_17,...,cmb_prb,cmb_clv_17,cmb_dsc_31,cmb_dsc_17,ipcc_clv,ipcc_dsc,area,oid,id_1,geometry
0,221682,12827,28,27,28-27,1,28.0,27.0,14.0,13.0,...,8.090109,14-13,Tierras Agrícolas - Pastizales,Tierras agrícolas - Pastizales y otra vegetaci...,15.0,CONV GL,0.41,8124,None,"POLYGON ((-103.00105 22.50049, -103.00105 22.5..."
1,68283,12703,27,3,27-3,1,27.0,3.0,13.0,2.0,...,16.683148,13-2,Pastizales - Bosque de Encino y Bosque de Galería,Pastizales y otra vegetación herbácea - Bosque...,9.0,CONV FL,0.70,4952,None,"POLYGON ((-102.99333 22.50049, -102.99333 22.5..."
2,68193,11427,14,27,14-27,1,14.0,27.0,7.0,13.0,...,17.413904,7-13,Matorral Crasicaule - Pastizales,Matorral alto denso - Pastizales y otra vegeta...,4.0,FL-GL,0.08,4862,None,"POLYGON ((-102.99480 22.50049, -102.99480 22.5..."
3,221758,12714,27,14,27-14,1,27.0,14.0,13.0,7.0,...,2.954326,13-7,Pastizales - Matorral Crasicaule,Pastizales y otra vegetación herbácea - Matorr...,9.0,CONV FL,0.20,8200,None,"POLYGON ((-102.99861 22.50049, -102.99828 22.5..."
4,68182,12703,27,3,27-3,1,27.0,3.0,13.0,2.0,...,16.683148,13-2,Pastizales - Bosque de Encino y Bosque de Galería,Pastizales y otra vegetación herbácea - Bosque...,9.0,CONV FL,0.11,4851,None,"POLYGON ((-102.96578 22.50042, -102.96559 22.5..."


In [34]:
gdf_reproj[list_name_attributes[0]][gdf_reproj[list_name_attributes[0]].isna()]

Series([], Name: t1_dsc_31, dtype: object)

In [35]:
gdf_reproj[list_name_attributes[1]][gdf_reproj[list_name_attributes[0]].isna()]

Series([], Name: t2_dsc_31, dtype: object)

In [36]:
gdf_reproj[list_name_attributes[2]][gdf_reproj[list_name_attributes[0]].isna()]

Series([], Name: t1_dsc_17, dtype: object)

In [37]:
gdf_reproj[list_name_attributes[2]][gdf_reproj[list_name_attributes[0]].isna()]

Series([], Name: t1_dsc_17, dtype: object)

**Normalize text**

In [38]:
import unicodedata

In [39]:
def normalize_name_classes(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII','ignore').decode('utf-8')

In [40]:
gdf_reproj[gdf_reproj.columns & list_name_attributes].head()

,t1_dsc_31,t2_dsc_31,t1_dsc_17,t2_dsc_17,cmb_dsc_31,cmb_dsc_17
0,Tierras Agrícolas,Pastizales,Tierras agrícolas,Pastizales y otra vegetación herbácea,Tierras Agrícolas - Pastizales,Tierras agrícolas - Pastizales y otra vegetaci...
1,Pastizales,Bosque de Encino y Bosque de Galería,Pastizales y otra vegetación herbácea,Bosques latifoliados,Pastizales - Bosque de Encino y Bosque de Galería,Pastizales y otra vegetación herbácea - Bosque...
2,Matorral Crasicaule,Pastizales,Matorral alto denso,Pastizales y otra vegetación herbácea,Matorral Crasicaule - Pastizales,Matorral alto denso - Pastizales y otra vegeta...
3,Pastizales,Matorral Crasicaule,Pastizales y otra vegetación herbácea,Matorral alto denso,Pastizales - Matorral Crasicaule,Pastizales y otra vegetación herbácea - Matorr...
4,Pastizales,Bosque de Encino y Bosque de Galería,Pastizales y otra vegetación herbácea,Bosques latifoliados,Pastizales - Bosque de Encino y Bosque de Galería,Pastizales y otra vegetación herbácea - Bosque...


In [41]:
gdf_reproj[gdf_reproj.columns & list_name_attributes] = gdf_reproj[gdf_reproj.columns & list_name_attributes].apply(lambda s: s.apply(normalize_name_classes))

In [42]:
gdf_reproj[gdf_reproj.columns & list_name_attributes].head()

,t1_dsc_31,t2_dsc_31,t1_dsc_17,t2_dsc_17,cmb_dsc_31,cmb_dsc_17
0,Tierras Agricolas,Pastizales,Tierras agricolas,Pastizales y otra vegetacion herbacea,Tierras Agricolas - Pastizales,Tierras agricolas - Pastizales y otra vegetaci...
1,Pastizales,Bosque de Encino y Bosque de Galeria,Pastizales y otra vegetacion herbacea,Bosques latifoliados,Pastizales - Bosque de Encino y Bosque de Galeria,Pastizales y otra vegetacion herbacea - Bosque...
2,Matorral Crasicaule,Pastizales,Matorral alto denso,Pastizales y otra vegetacion herbacea,Matorral Crasicaule - Pastizales,Matorral alto denso - Pastizales y otra vegeta...
3,Pastizales,Matorral Crasicaule,Pastizales y otra vegetacion herbacea,Matorral alto denso,Pastizales - Matorral Crasicaule,Pastizales y otra vegetacion herbacea - Matorr...
4,Pastizales,Bosque de Encino y Bosque de Galeria,Pastizales y otra vegetacion herbacea,Bosques latifoliados,Pastizales - Bosque de Encino y Bosque de Galeria,Pastizales y otra vegetacion herbacea - Bosque...
